In [1]:
import selenium
import datetime
from datetime import date, timedelta, datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
import pandas as pd
import csv
import math
import psycopg2

In [2]:
# 소요 시간 체크
global sec
def hms():
    global sec
    hours = sec // 3600
    sec = sec - hours*3600
    mu = sec // 60
    ss = sec - mu*60
    print('처리시간은 '+str(int(hours))+ '시간 '+ str(int(mu))+ '분 ' + str(int(ss)) + '초 걸렸습니다. '+ f"총 {int(sec)}초")


In [3]:
def review_check(__init__):    # 리뷰페이지로 이동하기
    browser.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
    browser.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
    try:
        time.sleep(2)
        browser.find_element(By.XPATH,'//*[@id="content"]/div/div[3]/div[3]/ul/li[2]/a').click()
        
    except:
        browser.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        browser.find_element(By.XPATH,'//*[@id="content"]/div/div[3]/div[3]/ul/li[2]/a').click()

In [4]:
def crawling_smart_store(__init__):
    rp_len = len(browser.find_elements(By.CLASS_NAME,'_2389dRohZq'))
    # 댓글 크롤링
    for i in range(rp_len) :
        try:
            a = browser.find_element(By.XPATH,'//*[@id="REVIEW"]/div/div[3]/div/div[2]/ul/li['+str(i+1)+']/div/div[1]/div/div[1]/div/div[1]/div[2]/div').text
            contents.append(a)
        except:
            pass

    # 날짜 크롤링
        try:
            dates.append(browser.find_element_by_xpath(f'//*[@id="REVIEW"]/div/div[3]/div/div[2]/ul/li[{i+1}]/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[2]/span').text)
            browser.implicitly_wait(1)
        except:
            dates.append('')
    
    # 별점 점수
        try:
            score.append(browser.find_element(By.XPATH,f'//*[@id="REVIEW"]/div/div[3]/div/div[2]/ul/li[{i+1}]/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[1]/em').text)
        except:
            score.append('')
    # 재구매
        try :
            if browser.find_element(By.XPATH,f'//*[@id="REVIEW"]/div/div[3]/div/div[2]/ul/li[{i+1}]/div/div[1]/div/div[1]/div/div/div[2]/div/span[1]').text == '재구매' :
                re_buy.append('재구매')
            elif browser.find_element(By.XPATH,f'//*[@id="REVIEW"]/div/div[3]/div/div[2]/ul/li[{i+1}]/div/div[1]/div/div[1]/div/div/div[2]/div/span[2]').text == '재구매' :
                re_buy.append('재구매')
            else :
                re_buy.append('0')
        except :
            re_buy.append('0')

    # 한달사용 후기
        try :
            if browser.find_element(By.XPATH,f'//*[@id="REVIEW"]/div/div[3]/div/div[2]/ul/li[{i+1}]/div/div[1]/div/div[1]/div/div/div[2]/div/span[1]').text == '한달사용기' :
                month_use.append('한달사용기')
            elif browser.find_element(By.XPATH,f'//*[@id="REVIEW"]/div/div[3]/div/div[2]/ul/li[{i+1}]/div/div[1]/div/div[1]/div/div/div[2]/div/span[2]').text == '한달사용기' :
                month_use.append('한달사용기')
            else :
                month_use.append('0')
        except :
            month_use.append('0')
            
    # 종류
        try : 
            category.append(browser.find_element_by_xpath(f'//*[@id="REVIEW"]/div/div[3]/div/div[2]/ul/li[{i+1}]/div/div/div/div[1]/div/div/div[1]/div[2]/div[3]/div/button/span').text)
        except :
            category.append('0')

In [5]:
# DB 접속 (postgresql)
conn_str = "host=localhost dbname=mydb user=postgres password=admin port = 5432"
try :
    conn = psycopg2.connect(conn_str)
    print("=== 접속 성공 =====")
except psycopg2. DatabaseError as db_err:
    print("접속오류 !!")
    print(db_err)

=== 접속 성공 =====


In [6]:
# 테이블 생성
cur = conn.cursor()  # conn = 위에 나오듯 포스트그리 커넥션으로 연결을 말한다.
try:
    cur.execute('create table if not exists test(num serial, review varchar(900), dates varchar(100), rate varchar(100), re_buy varchar(100), month_use varchar(100), category varchar(100));')
    conn.commit()             
    print('생성 성공')
except :
    conn.rollback()
    print('---DB error')

생성 성공


In [7]:
# DB에 각 페이지의 댓글들 저장하기
def into_DB():
    for i in range(rp_len):
        try :                   # 수집한 댓글들은 모두 str이라 %s로, num은 값을 int로 하여도 %d로 저장되지 않아 %s로 하였다.
            cur.execute('INSERT INTO test (num,review, dates, rate, re_buy, month_use, category) VALUES (%s, %s, %s, %s, %s, %s, %s);',
            (int((z*20)+i+1),contents[(z*20)+i],dates[(z*20)+i],score[(z*20)+i],re_buy[(z*20)+i],month_use[(z*20)+i],category[(z*20)+i]))
            conn.commit()
            print("----- insert ok--")                                     
        except :
            conn. rollback()
            print("--- DB error")

In [8]:
# DB에 담지못한 나머지 페이지의 댓글들 저장하기
def into_DB_rest():
    for i in range(rp_len):
        try :                   # 수집한 댓글들은 모두 str이라 %s로, num은 값을 int로 하여도 %d로 저장되지 않아 %s로 하였다.
            cur.execute('INSERT INTO test (num,review, dates, rate, re_buy, month_use, category) VALUES (%s, %s, %s, %s, %s, %s, %s);',
            (int((page*20)+1+i),contents[(page*20)+i],dates[(page*20)+i],score[(page*20)+i],re_buy[(page*20)+i],month_use[(page*20)+i],category[(page*20)+i]))
            conn.commit()
            print("----- insert ok--")                                     
        except :
            conn. rollback()
            print("--- DB error")

In [9]:
def login():
    if len(browser.find_elements(By.CLASS_NAME, 'id_pw_wrap'))>0:
        id = input()
        pw = input()
        browser.execute_script("document.getElementsByName('id')[0].value = \'" + id + "\'")
        time.sleep(1)
        browser.execute_script("document.getElementsByName('pw')[0].value=\'" + pw + "\'")
        time.sleep(1)
        browser.find_element_by_css_selector('#log\.login').click()
        time.sleep(1)   
    else:
        pass

In [11]:
# 스마트스토어 크롤링
browser = webdriver.Chrome()
browser.implicitly_wait(5)
browser.maximize_window() # 브라우저 화면 최대화
browser.implicitly_wait(10)
time.sleep(2)

# 데이터 크롤링 시작시간
print('데이터 수집 시작' , datetime.today().strftime("%Y/%m/%d %H:%M:%S"))
start = time.time()

# 페이지 get + 2022.5.23일 스마트스토어 업데이트 확인에 따라 크롤링 코드 추가
browser.get('https://smartstore.naver.com/로 시작하는 url을 넣어주세요')     # https://smartstore.naver.com/로 시작하는 url을 넣어주세요
browser.implicitly_wait(10)
time.sleep(2)


# 로그인 있다면 하기
login()


# 리뷰페이지로 이동하기
review_check(3)
browser.implicitly_wait(5)
browser.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
time.sleep(2)
browser.find_element(By.XPATH,'//*[@id="REVIEW"]/div/div[3]/div/div[1]/div[1]/ul/li[2]').click()
time.sleep(2)

# 댓글, 날짜, 별점, 재구매여부, 한달사용후기, 구매칫솔종류
contents = []
dates = []
score = []
re_buy = []
category = []
month_use = []


# 댓글페이지 계산. 2만건 이상은 빈페이지만 나오는 제한이 있으므로, 2만건 이상은 페이지를 지정함
review_range = browser.find_element(By.XPATH,'//*[@id="content"]/div/div[2]/div[1]/div[2]/div[1]/a').text
review_range = int(review_range.replace(',',''))
if review_range >= 20000:
    page = 1000
elif review_range < 20000:
    page = (review_range // 20)


# p는 페이지 넘기기 위한 번호 임의 생성
p = 2
for z in range(page): 
    print("{0}페이지 중 {1}번째 페이지".format(page,z+1))
    rp_len = len(browser.find_elements(By.CLASS_NAME,'_2389dRohZq'))

    # 댓글 수집
    crawling_smart_store(z)
    
    # DB저장
    into_DB()
    
    
    # 다음 페이지로 이동       
    p +=1   
    
    
    # 페이지 넘어간 이후 계속해서 click하기 위해 작성
    try:
        browser.find_element_by_xpath('//*[@id="REVIEW"]/div/div[3]/div/div[2]/div/div/a['+ str(p) +']').click()
        if p == 12:
            p = 2
        browser.implicitly_wait(5)
        time.sleep(random.uniform(0.5,1.5))
    except:
        browser.find_element_by_xpath('//*[@id="REVIEW"]/div/div[3]/div/div[2]/div/div/a['+ str(p) +']').text == ''


# 마지막 페이지의 댓글 수집하기 
if review_range < 20000:                # 마지막으로 넘긴 페이지 나머지 댓글 가져오기
    if review_range % 20 > 0:
        print("총 {0}페이지 + {0}번째 페이지의 나머지 댓글".format(page+1))
        rp_len = len(browser.find_elements(By.CLASS_NAME,'_2389dRohZq'))
        # 댓글 수집
        crawling_smart_store(1)
        # DB 저장
        into_DB_rest()
else:                                   # 페이지를 넘어가지않고 다 가져온 경우 + 1000페이지를 넘어가 빈페이지만 나온 경우 
    print("총 {0}페이지".format(page))
    pass


# 데이터 크롤링 완료 + 소요시간 확인
print('완료', datetime.today().strftime("%Y/%m/%d %H:%M:%S"))
end = time.time()
sec = (end - start)
hms()
browser.quit()

# 데이터 프레임화
df = pd.DataFrame({'내용':contents, '날짜':dates,'별점':score,'재구매':re_buy,'한달사용기':month_use,'종류':category})

데이터 수집 시작 2022/06/02 10:13:31


c:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  if __name__ == "__main__":


544페이지 중 1번째 페이지


c:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  del sys.path[0]
c:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:47: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


In [ ]:
# 페이지당 소요시간 확인. 인터넷과 컴퓨터 성능에 영향을 받음
if review_range < 20000:
    if review_range % 20 > 0:
        print(f'페이지당 평균 {int(sec / (page+1))}초 사용')
else:
    print(f'페이지당 평균 {int(sec / (page))}초 사용')
    pass

페이지당 평균 33초 사용


In [ ]:
# 위와 통합시 간혹 저장되지않는 경우 확인. 따로 빼서 할 시 저장됨.
df.to_csv('db테스트한다구.csv')

In [ ]:
df

,내용,날짜,별점,재구매,한달사용기,종류
0,조금 무겁긴해도 요기나게 쓰고있어요,22.05.16.,5,0,0,컬러: 베이지
1,여자가가지고다니기에살짝무거운것같은..\n야구장갈때쓸려고구매햇는데..못쓰게하더라구요ㅡ...,22.05.06.,5,0,0,컬러: 베이지
2,빠르게 배송되어 야구장에서 사용했는데 조금 흔들리고 작긴 했지만 그런데로 쓸만하네요...,22.04.27.,4,0,0,컬러: 베이지
3,마감이 별로네요. 그리고 상판이 단단하지 않고 너무 쉽게 휘는데 써봐야 괜찮은지 알...,22.04.13.,3,0,0,컬러: 베이지
4,배송은 좀 느리지만 제품은 좋네요,22.04.13.,5,0,0,컬러: 베이지
...,...,...,...,...,...,...
58,바로 야구장가서 썼어요^^\n조립분리간편해서 좋아요♡,19.07.21.,5,0,0,0
59,한달사용기그냥저냥 만족합니다.,19.07.15.,3,0,한달사용기,0
60,배송빠르고 제품도 좋아요,19.05.02.,5,0,0,0
61,너무간편합니다! 쇼파위에서 잘쓰고있어요! 최고입니다 번창하세요^^,19.03.06.,5,0,0,0
